<a href="https://colab.research.google.com/github/jorgcham/Porfolio/blob/main/fundamental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 0. STYLING & CONFIGURATION
# ==========================================
class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

class SectorBenchmarks:
    """Granular sector benchmarks (2025/2026 estimates)"""
    SECTOR_METRICS = {
        'Technology': {'pe': (20, 35), 'ev_ebitda': (15, 25), 'net_margin': 15, 'roic': 15},
        'Communication Services': {'pe': (15, 25), 'ev_ebitda': (10, 18), 'net_margin': 12, 'roic': 12},
        'Consumer Cyclical': {'pe': (15, 25), 'ev_ebitda': (10, 16), 'net_margin': 8, 'roic': 10},
        'Consumer Defensive': {'pe': (18, 24), 'ev_ebitda': (12, 17), 'net_margin': 6, 'roic': 10},
        'Financial Services': {'pe': (10, 16), 'ev_ebitda': (8, 14), 'net_margin': 15, 'roic': 8},
        'Healthcare': {'pe': (18, 26), 'ev_ebitda': (12, 18), 'net_margin': 10, 'roic': 12},
        'Energy': {'pe': (8, 14), 'ev_ebitda': (5, 9), 'net_margin': 8, 'roic': 10},
        'Industrials': {'pe': (16, 22), 'ev_ebitda': (10, 15), 'net_margin': 7, 'roic': 10},
        'Basic Materials': {'pe': (12, 18), 'ev_ebitda': (6, 10), 'net_margin': 8, 'roic': 9},
        'Utilities': {'pe': (16, 20), 'ev_ebitda': (9, 13), 'net_margin': 9, 'roic': 5},
        'Real Estate': {'pe': (25, 45), 'ev_ebitda': (15, 22), 'net_margin': 15, 'roic': 5}
    }
    DEFAULT = {'pe': (15, 25), 'ev_ebitda': (10, 15), 'net_margin': 10, 'roic': 10}

class DataHelper:
    @staticmethod
    def get_val(df, keys, idx=0):
        """Safely retrieves value searching multiple keys"""
        for k in keys:
            if k in df.index and len(df.loc[k]) > idx:
                v = df.loc[k].iloc[idx]
                return float(v) if not pd.isna(v) else 0.0
        return 0.0

# ==========================================
# 1. ENHANCED ANALYSIS ENGINE
# ==========================================
class HedgeFundAnalyst:
    def __init__(self, ticker):
        self.ticker = ticker.upper()
        print(f"\n{Colors.CYAN}🚀 Starting analysis engine for {self.ticker}...{Colors.ENDC}")
        try:
            self.stock = yf.Ticker(ticker)
            self.info = self.stock.info
            self.income = self.stock.financials
            self.balance = self.stock.balance_sheet
            self.cashflow = self.stock.cashflow

            # Load historical data
            self.hist_1y = self.stock.history(period="1y")
            self.hist_3y = self.stock.history(period="3y")

            if self.income.empty: raise ValueError("Data unavailable (possible delisting or ticker error)")

            self.sector = self.info.get('sector', 'General')
            self.bench = SectorBenchmarks.SECTOR_METRICS.get(self.sector, SectorBenchmarks.DEFAULT)
        except Exception as e:
            raise ValueError(f"Critical error: {e}")

    def _div(self, n, d): return n/d if d else 0.0

    # --- 1. WALL STREET CONSENSUS ---
    def get_consensus(self):
        rec = self.info.get('recommendationKey', 'none').upper().replace('_', ' ')
        target = self.info.get('targetMeanPrice', 0)
        curr = self.info.get('currentPrice', 0)
        upside = ((target - curr)/curr*100) if curr else 0
        num_analysts = self.info.get('numberOfAnalystOpinions', 0)

        # Target dispersion
        target_high = self.info.get('targetHighPrice', 0)
        target_low = self.info.get('targetLowPrice', 0)
        dispersion = ((target_high - target_low) / target * 100) if target else 0

        return rec, target, upside, num_analysts, target_low, target_high, dispersion

    # --- 2. OWNERSHIP STRUCTURE ---
    def get_ownership(self):
        inst_own = self.info.get('heldPercentInstitutions', 0) * 100
        insider_own = self.info.get('heldPercentInsiders', 0) * 100
        short_float = self.info.get('shortPercentOfFloat', 0) * 100
        return inst_own, insider_own, short_float

    # --- 3. DEEP VALUATION ---
    def get_valuation(self):
        pe = self.info.get('trailingPE', 0)
        fwd_pe = self.info.get('forwardPE', 0)
        peg = self.info.get('pegRatio', 0)
        ps = self.info.get('priceToSalesTrailing12Months', 0)

        # P/E benchmarking
        pe_status = "OK"
        if pe > 0:
            if pe < self.bench['pe'][0]: pe_status = "CHEAP"
            elif pe > self.bench['pe'][1]: pe_status = "EXPENSIVE"

        return {'pe': pe, 'fwd_pe': fwd_pe, 'peg': peg, 'ps': ps, 'status': pe_status}

    # --- 4. QUALITY (SIMPLIFIED FORENSIC 0-5) ---
    def get_quality_score(self):
        score = 0
        checks = []

        # 1. Net profitability
        ni = DataHelper.get_val(self.income, ['Net Income', 'Net Income Common Stockholders'])
        if ni > 0: score += 1

        # 2. Operating cash flow
        cfo = DataHelper.get_val(self.cashflow, ['Operating Cash Flow', 'Total Cash From Operating Activities'])
        if cfo > 0: score += 1

        # 3. Earnings quality
        if cfo > ni: score += 1
        else: checks.append(" Earnings > Cash (Low Quality)")

        # 4. Debt
        debt_now = DataHelper.get_val(self.balance, ['Total Debt', 'Long Term Debt'])
        debt_prev = DataHelper.get_val(self.balance, ['Total Debt', 'Long Term Debt'], 1)
        if debt_now <= debt_prev: score += 1
        else: checks.append("Debt increasing")

        # 5. Gross margins
        gm_now = self._div(DataHelper.get_val(self.income, ['Gross Profit']), DataHelper.get_val(self.income, ['Total Revenue']))
        gm_prev = self._div(DataHelper.get_val(self.income, ['Gross Profit'], 1), DataHelper.get_val(self.income, ['Total Revenue'], 1))
        if gm_now >= gm_prev: score += 1

        return score, checks

    # --- RISK METRICS ---
    def get_risk_metrics(self):
        """Calculates volatility, beta, max drawdown"""
        if self.hist_1y.empty:
            return {'volatility': 0, 'beta': 0, 'max_drawdown': 0, 'sharpe': 0}

        # Daily returns
        returns = self.hist_1y['Close'].pct_change().dropna()

        # Annualized volatility
        volatility = returns.std() * np.sqrt(252) * 100

        # Beta (vs S&P 500)
        try:
            spy = yf.Ticker("SPY").history(period="1y")['Close'].pct_change().dropna()
            # Align dates
            aligned = pd.concat([returns, spy], axis=1, join='inner')
            aligned.columns = ['stock', 'market']
            covariance = aligned.cov().iloc[0, 1]
            market_var = aligned['market'].var()
            beta = covariance / market_var if market_var != 0 else 0
        except:
            beta = 0

        # Max drawdown
        cumulative = (1 + returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min() * 100

        # Sharpe Ratio (assuming risk-free = 4%)
        risk_free = 0.04
        excess_return = returns.mean() * 252 - risk_free
        sharpe = excess_return / (returns.std() * np.sqrt(252)) if returns.std() != 0 else 0

        return {
            'volatility': volatility,
            'beta': beta,
            'max_drawdown': max_drawdown,
            'sharpe': sharpe
        }

    # --- CORPORATE EVENTS ---
    def get_corporate_events(self):
        """Detects upcoming earnings, dividends, splits"""
        events = []

        # Earnings date
        earnings_dates = self.info.get('earningsTimestamp', None)
        if earnings_dates:
            try:
                earnings_dt = datetime.fromtimestamp(earnings_dates)
                days_to_earnings = (earnings_dt - datetime.now()).days
                if -30 <= days_to_earnings <= 30:
                    events.append(f" Earnings in {days_to_earnings} days ({earnings_dt.strftime('%Y-%m-%d')})")
            except:
                pass

        # Ex-Dividend Date
        ex_div = self.info.get('exDividendDate', None)
        if ex_div:
            try:
                ex_div_dt = datetime.fromtimestamp(ex_div)
                days_to_div = (ex_div_dt - datetime.now()).days
                if 0 <= days_to_div <= 30:
                    div_rate = self.info.get('dividendRate', 0)
                    events.append(f" Ex-Dividend in {days_to_div} days (${div_rate}/share)")
            except:
                pass

        # Insider Purchases (last 6 weeks)
        try:
            insider_txns = self.stock.insider_transactions
            if not insider_txns.empty:
                recent = insider_txns[insider_txns['Start Date'] > (datetime.now() - timedelta(days=42))]
                buys = recent[recent['Transaction'] == 'Purchase']
                if len(buys) > 0:
                    events.append(f" {len(buys)} insider purchases (last 6 weeks)")
        except:
            pass

        return events

    # --- SIMPLE BACKTESTING ---
    def backtest_strategy(self):
        """Simulates what would have happened 1 year ago with this methodology"""
        if self.hist_3y.empty or len(self.hist_3y) < 252:
            return None

        try:
            # Point 1 year ago
            one_year_ago_idx = -252
            price_1y_ago = self.hist_3y['Close'].iloc[one_year_ago_idx]
            price_now = self.hist_3y['Close'].iloc[-1]
            actual_return = ((price_now - price_1y_ago) / price_1y_ago) * 100

            # Simulate score 1 year ago (simplified - price data only)
            # In production you'd do this with historical fundamentals
            hist_volatility = self.hist_3y['Close'].iloc[one_year_ago_idx-252:one_year_ago_idx].pct_change().std() * np.sqrt(252) * 100

            # Simple prediction: if volatility < 30% and YTD returns positive → BUY
            ytd_return_then = ((price_1y_ago - self.hist_3y['Close'].iloc[one_year_ago_idx-252]) /
                              self.hist_3y['Close'].iloc[one_year_ago_idx-252]) * 100

            predicted_signal = "BUY" if (hist_volatility < 35 and ytd_return_then > 0) else "HOLD"

            return {
                'predicted_signal': predicted_signal,
                'actual_return': actual_return,
                'would_have_worked': (predicted_signal == "BUY" and actual_return > 0)
            }
        except:
            return None

    # --- DETAILED SCORING BREAKDOWN ---
    def calculate_verdict_detailed(self, val, qual_score, upside_wallstreet, short_float, risk_metrics):
        breakdown = {}
        points = 0

        # VALUATION (40 pts)
        val_points = 0
        if val['peg'] > 0 and val['peg'] < 1.5:
            val_points += 15
            breakdown['peg_bonus'] = 15
        elif val['peg'] > 0:
            breakdown['peg_bonus'] = 0

        if val['pe'] > 0 and val['status'] == "CHEAP":
            val_points += 15
            breakdown['pe_status'] = 15
        elif val['status'] == "OK":
            val_points += 10
            breakdown['pe_status'] = 10
        else:
            breakdown['pe_status'] = 0

        if val['ps'] < 10:
            val_points += 10
            breakdown['ps_check'] = 10
        else:
            breakdown['ps_check'] = 0

        breakdown['valuation_total'] = val_points
        points += val_points

        # QUALITY (30 pts)
        qual_points = 0
        if qual_score >= 4:
            qual_points = 30
        elif qual_score >= 3:
            qual_points = 15
        breakdown['quality_score'] = qual_points
        points += qual_points

        # SENTIMENT (20 pts)
        sent_points = 0
        if upside_wallstreet > 10:
            sent_points += 15
            breakdown['wallstreet_upside'] = 15
        elif upside_wallstreet > 0:
            sent_points += 5
            breakdown['wallstreet_upside'] = 5
        else:
            breakdown['wallstreet_upside'] = 0

        if short_float < 5:
            sent_points += 5
            breakdown['short_interest'] = 5
        elif short_float < 10:
            sent_points += 2
            breakdown['short_interest'] = 2
        else:
            breakdown['short_interest'] = 0

        breakdown['sentiment_total'] = sent_points
        points += sent_points

        # RISK (10 pts)
        risk_points = 0
        if risk_metrics['sharpe'] > 1.0:
            risk_points += 5
            breakdown['sharpe_bonus'] = 5
        elif risk_metrics['sharpe'] > 0.5:
            risk_points += 2
            breakdown['sharpe_bonus'] = 2
        else:
            breakdown['sharpe_bonus'] = 0

        if risk_metrics['max_drawdown'] > -25:
            risk_points += 5
            breakdown['drawdown_bonus'] = 5
        elif risk_metrics['max_drawdown'] > -40:
            risk_points += 2
            breakdown['drawdown_bonus'] = 2
        else:
            breakdown['drawdown_bonus'] = 0

        breakdown['risk_total'] = risk_points
        points += risk_points

        # VERDICT
        verdict = "SELL"
        if points >= 75: verdict = "STRONG BUY"
        elif points >= 60: verdict = "BUY"
        elif points >= 40: verdict = "HOLD"

        return verdict, points, breakdown

    # --- GENERATE ENHANCED REPORT ---
    def generate_report(self):
        # Gather data
        cons_rec, cons_target, cons_upside, cons_num, target_low, target_high, dispersion = self.get_consensus()
        inst, insider, short = self.get_ownership()
        val = self.get_valuation()
        qual_score, qual_checks = self.get_quality_score()
        risk_metrics = self.get_risk_metrics()
        events = self.get_corporate_events()
        backtest = self.backtest_strategy()

        # Calculate detailed verdict
        algo_verdict, algo_score, breakdown = self.calculate_verdict_detailed(
            val, qual_score, cons_upside, short, risk_metrics
        )

        # Colors
        c_verdict = Colors.GREEN if algo_score >= 60 else Colors.FAIL if algo_score < 40 else Colors.WARNING
        c_upside = Colors.GREEN if cons_upside > 0 else Colors.FAIL

        print("\n" + "="*80)
        print(f"{Colors.HEADER}  HEDGE FUND TERMINAL: {self.ticker} ({self.sector}){Colors.ENDC}")
        print("="*80)

        # SECTION 1: THE VERDICT
        print(f"\n{Colors.BOLD}╔══════════════════════════════════════════════════════════════════════════════╗{Colors.ENDC}")
        print(f"{Colors.BOLD}║  ALGORITHM VERDICT:     {c_verdict}{algo_verdict:15}{Colors.ENDC} (Score: {algo_score}/100){' '*19}║{Colors.ENDC}")
        print(f"{Colors.BOLD}║  WALL ST CONSENSUS:     {cons_rec:15} (Based on {cons_num} analysts){' '*14}║{Colors.ENDC}")
        print(f"{Colors.BOLD}╚══════════════════════════════════════════════════════════════════════════════╝{Colors.ENDC}")

        print(f"\n{Colors.BOLD}>>> KEY DRIVERS:{Colors.ENDC}")
        print(f"   1. Upside Potential:    {c_upside}{cons_upside:+.2f}%{Colors.ENDC} (Target: ${cons_target:.2f})")

        # Analyst dispersion
        if dispersion > 50:
            print(f"      {Colors.WARNING} HIGH DISPERSION:{Colors.ENDC} Analysts range ${target_low:.2f} to ${target_high:.2f} ({dispersion:.0f}% spread)")

        pe_display = f"{val['pe']:.1f}x" if val['pe'] else "N/A"
        print(f"   2. Valuation Status:    {val['status']} vs Sector (P/E {pe_display} vs {self.bench['pe'][0]}-{self.bench['pe'][1]}x)")
        print(f"   3. Quality Score:       {qual_score}/5 (Simplified Piotroski)")

        if short > 10:
             print(f"   4. {Colors.FAIL} SHORT ALERT:{Colors.ENDC}      {short:.2f}% of float is betting against")
        else:
             print(f"   4. Short Risk:          Low ({short:.2f}%)")

        # RISK METRICS
        print(f"\n{Colors.BOLD}[RISK PROFILE]{Colors.ENDC}")
        print(f"   Volatility (1Y):     {risk_metrics['volatility']:.1f}% annualized")

        beta_color = Colors.GREEN if 0.8 <= risk_metrics['beta'] <= 1.2 else Colors.WARNING
        print(f"   Beta vs S&P 500:     {beta_color}{risk_metrics['beta']:.2f}{Colors.ENDC} ", end="")
        if risk_metrics['beta'] > 1.5:
            print("(VERY AGGRESSIVE)")
        elif risk_metrics['beta'] < 0.5:
            print("(DEFENSIVE)")
        else:
            print("(NEUTRAL)")

        dd_color = Colors.GREEN if risk_metrics['max_drawdown'] > -20 else Colors.FAIL
        print(f"   Max Drawdown:        {dd_color}{risk_metrics['max_drawdown']:.1f}%{Colors.ENDC}")

        sharpe_color = Colors.GREEN if risk_metrics['sharpe'] > 1 else Colors.WARNING
        print(f"   Sharpe Ratio:        {sharpe_color}{risk_metrics['sharpe']:.2f}{Colors.ENDC}")

        # CORPORATE EVENTS
        if events:
            print(f"\n{Colors.BOLD}[UPCOMING EVENTS]{Colors.ENDC}")
            for event in events:
                print(f"   {event}")

        # SCORE BREAKDOWN
        print(f"\n{Colors.BOLD}[SCORE BREAKDOWN: {algo_score}/100]{Colors.ENDC}")
        print(f"   • Valuation ({breakdown['valuation_total']}/40 pts):")
        print(f"      - PEG Ratio: {breakdown.get('peg_bonus', 0)} pts")
        print(f"      - P/E vs Sector: {breakdown['pe_status']} pts")
        print(f"      - P/S Check: {breakdown['ps_check']} pts")
        print(f"   • Quality ({breakdown['quality_score']}/30 pts): Score {qual_score}/5")
        print(f"   • Sentiment ({breakdown['sentiment_total']}/20 pts):")
        print(f"      - Wall Street Upside: {breakdown['wallstreet_upside']} pts")
        print(f"      - Short Interest: {breakdown['short_interest']} pts")
        print(f"   • Risk ({breakdown['risk_total']}/10 pts):")
        print(f"      - Sharpe Ratio: {breakdown['sharpe_bonus']} pts")
        print(f"      - Drawdown Control: {breakdown['drawdown_bonus']} pts")

        # BACKTESTING
        if backtest:
            print(f"\n{Colors.BOLD}[BACKTESTING: What if you invested 1 year ago?]{Colors.ENDC}")
            print(f"   Predicted Signal:    {backtest['predicted_signal']}")
            print(f"   Actual Return:       {backtest['actual_return']:+.2f}%")

            if backtest['would_have_worked']:
                print(f"   {Colors.GREEN} Strategy WORKED{Colors.ENDC}")
            else:
                print(f"   {Colors.FAIL} Strategy did NOT work (learn from this){Colors.ENDC}")

        # DATA ROOM
        print(f"\n{Colors.BOLD}[DATA ROOM] DEEP ANALYSIS{Colors.ENDC}")
        print("-" * 75)
        print(f"{'METRIC':<25} | {'VALUE':<15} | {'SECTOR BENCHMARK':<20} | {'STATUS':<10}")
        print("-" * 75)

        def print_row(name, val, bench_tuple, format_str="{:.2f}"):
            if val is None or val == 0:
                print(f"{name:<25} | {'N/A':<15} | {'-':<20} | {'-':<10}")
                return

            v_fmt = format_str.format(val)
            low, high = bench_tuple
            status = "NEUTRAL"
            color = Colors.BLUE

            if "Margin" in name or "ROIC" in name:
                if val < low: status = "LOW"; color = Colors.FAIL
                elif val > high: status = "TOP"; color = Colors.GREEN
                else: status = "OK"; color = Colors.BLUE
            elif "P/E" in name or "EV/" in name:
                if val < low: status = "CHEAP"; color = Colors.GREEN
                elif val > high: status = "EXPENSIVE"; color = Colors.FAIL
                else: status = "OK"; color = Colors.BLUE

            print(f"{name:<25} | {v_fmt:<15} | {low}-{high:<19} | {color}{status:<10}{Colors.ENDC}")

        print_row("P/E Ratio (TTM)", val['pe'], self.bench['pe'])
        print_row("Forward P/E", val['fwd_pe'], self.bench['pe'])
        print_row("EV / EBITDA", self.info.get('enterpriseToEbitda', 0), self.bench['ev_ebitda'])
        print_row("Net Margin (%)", self.info.get('profitMargins', 0)*100, (self.bench['net_margin'], 100))

        roe = self.info.get('returnOnEquity', 0)
        print_row("ROIC/ROE (%)", roe*100, (self.bench['roic'], 100))

        print("-" * 75)

        # OWNERSHIP STRUCTURE
        print(f"\n{Colors.BOLD}[OWNERSHIP STRUCTURE]{Colors.ENDC}")
        print(f"   Institutions:    {inst:.1f}%  (Funds, Banks)")
        print(f"   Insiders:        {insider:.1f}%  (Owners, CEO)")
        if insider > 10: print(f"   {Colors.GREEN}✨ HIGH ALIGNMENT: Owners have significant skin in the game{Colors.ENDC}")
        if inst > 80: print(f"     INSTITUTIONAL STOCK: Heavily controlled by large funds")

        if qual_checks:
            print(f"\n{Colors.BOLD} RISK NOTES:{Colors.ENDC} " + ", ".join(qual_checks))

        print("\n" + "="*80 + "\n")

if __name__ == "__main__":
    t = input("Ticker (e.g. AMZN, TSLA): ")
    try:
        analyst = HedgeFundAnalyst(t)
        analyst.generate_report()
    except Exception as e:
        print(f"Error: {e}")

Ticker (e.g. AMZN, TSLA): BAB.L

🚀 Starting analysis engine for BAB.L...

  HEDGE FUND TERMINAL: BAB.L (Industrials)

╔══════════════════════════════════════════════════════════════════════════════╗
║  ALGORITHM VERDICT:     HOLD            (Score: 55/100)                   ║
║  WALL ST CONSENSUS:     BUY             (Based on 9 analysts)              ║
╚══════════════════════════════════════════════════════════════════════════════╝

>>> KEY DRIVERS:
   1. Upside Potential:    -9.22% (Target: $1336.32)
   2. Valuation Status:    EXPENSIVE vs Sector (P/E 26.3x vs 16-22x)
   3. Quality Score:       4/5 (Simplified Piotroski)
   4. Short Risk:          Low (0.00%)

[RISK PROFILE]
   Volatility (1Y):     35.3% annualized
   Beta vs S&P 500:     nan (NEUTRAL)
   Max Drawdown:        -18.1%
   Sharpe Ratio:        3.12

[SCORE BREAKDOWN: 55/100]
   • Valuation (10/40 pts):
      - PEG Ratio: 0 pts
      - P/E vs Sector: 0 pts
      - P/S Check: 10 pts
   • Quality (30/30 pts): Score 4/5
   •